<a href="https://colab.research.google.com/github/rzendonadi/dataviz/blob/main/scraping_%2B_kepler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
#installing libs
#!pip install geopandas
#!pip install geobr
#!pip install keplergl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 54.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jedi-0.18.2-py2.py3-none-any.whl (1.6 MB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922343 sha256=8e9f3670f871a8e57bad8d27c07db2940a5b3c7c25cd823dfdf4e6143c587ae1
  Stored in directory: /root/.cache/pip/wheels/15/7f/48/aa4980536f7c9fbc783b2a191a579e73e7f74c41afc3f33c61
Successfully built keplergl


In [ ]:
#importing
import geopandas as gpd
import geobr
import pandas as pd
import time
import shapely

In [62]:
#defining geographic AOI
dict_states = dict_states = {'uf':['RJ', 'SP']}
states = pd.DataFrame(dict_states)

In [63]:
#retrieving json from df
s = time.time()
aoi = pd.DataFrame()
for index, row in states.iterrows():
    t = geobr.read_municipality(code_muni = (row['uf']), year=2020)
    if len(t)>0:
        aoi = aoi.append((pd.DataFrame(t)))
print(time.time() - s)

0.024387836456298828


In [64]:
#conversion
aoi['geom'] = aoi['geometry'].astype(str)
aoi.drop(columns=['geometry'])

,code_muni,name_muni,code_state,abbrev_state,name_state,code_region,name_region,geom
0,3300100.0,Angra Dos Reis,33.0,RJ,Rio de Janeiro,3.0,Sudeste,MULTIPOLYGON (((-44.15109253499999 -23.2160218...
1,3300159.0,Aperibé,33.0,RJ,Rio de Janeiro,3.0,Sudeste,MULTIPOLYGON (((-42.125357236 -21.601551931000...
2,3300209.0,Araruama,33.0,RJ,Rio de Janeiro,3.0,Sudeste,MULTIPOLYGON (((-42.18387478099999 -22.5514357...
3,3300225.0,Areal,33.0,RJ,Rio de Janeiro,3.0,Sudeste,MULTIPOLYGON (((-43.03715950299999 -22.1642924...
4,3300233.0,Armação Dos Búzios,33.0,RJ,Rio de Janeiro,3.0,Sudeste,MULTIPOLYGON (((-41.91855270400001 -22.8157420...
...,...,...,...,...,...,...,...,...
640,3557006.0,Votorantim,35.0,SP,São Paulo,3.0,Sudeste,MULTIPOLYGON (((-47.37298537299999 -23.5220010...
641,3557105.0,Votuporanga,35.0,SP,São Paulo,3.0,Sudeste,MULTIPOLYGON (((-50.021205611 -20.333935066000...
642,3557154.0,Zacarias,35.0,SP,São Paulo,3.0,Sudeste,MULTIPOLYGON (((-49.99274663599999 -21.0388351...
643,3557204.0,Chavantes,35.0,SP,São Paulo,3.0,Sudeste,MULTIPOLYGON (((-49.68630378999999 -22.9378735...


In [65]:
#recognizing geometry
df = gpd.GeoDataFrame(aoi, crs="epsg:4326", geometry = aoi['geom'].map(shapely.wkt.loads))

/usr/local/lib/python3.9/dist-packages/geopandas/array.py:93: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [66]:
#creating lat, lng from centroid
df['lng'] = (df.centroid.x).astype(str)
df['lat'] = (df.centroid.y).astype(str)

/usr/local/lib/python3.9/dist-packages/geopandas/array.py:546: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return _unary_geo("centroid", self)
/usr/local/lib/python3.9/dist-packages/geopandas/geoseries.py:128: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  data = from_shapely(s.values)


In [53]:
#url_sample = 'https://mcd-latam-landings-backend-l.gigigoapps.com/restaurants/near?country=BR&lat=-23.599378707752425&lng=-46.73918803099497&radius=50000&limit=100'

In [67]:
#url for each centroid
s = time.time()
for index, row in df.iterrows():
    df['url'] = "https://mcd-latam-landings-backend-l.gigigoapps.com/restaurants/near?country=BR&lat=" + df['lat'] + '&lng=' + df['lng'] + "&radius=50000&limit=100"
print(time.time() - s)

0.5910608768463135


In [68]:
#retrieving json from df
s = time.time()
#z = []
dz = pd.DataFrame(columns = ['_id', 'siglas', 'name', 'city', 'neighborhood', 'address', 'cep', 'services', 'country', 'active', 'longitude', 'latitude', 'createdAt', 'updatedAt', '__v', 'id', 'distance'])
for index, row in df.iterrows():
    t = pd.read_json(row['url'])
    if len(t)>0:
        dz = dz.append((pd.DataFrame(t)[['_id', 'siglas', 'name', 'city', 'neighborhood', 'address', 'cep', 'services', 'country', 'active', 'longitude', 'latitude', 'createdAt', 'updatedAt', '__v', 'id', 'distance']]))
        #z.append(t)
print(time.time() - s)

53.313822507858276


In [75]:
#split nested column to multiple
#dz[['breakfast', 'mcCafe', 'driveThru', 'dcOut', 'dcIn', 'mcDelivery', 'timeExtended', 'mcParty', 'playPlace', 'parking', 'wifi', 'wheelchairAccess', 'dessertCenter', 'shoppingCenter']] = dz['services'].str.split(',', expand=True)

ValueError: ignored

In [87]:
from keplergl import KeplerGl
# Load a map with data and config and height
map_1 = KeplerGl(height=400, data={"data_1": dz})
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_1':                          _id siglas                      name  \
0   60211e88fcd1b840…

In [86]:
map_1.save_to_html(file_name='first_map.html')

Map saved to first_map.html!
